In [4]:
#課題5
import mpmath

print('mpmath.libmp.BACKEND = ',mpmath.libmp.BACKEND)

mpmath.mp.dps=350
x=[mpmath.mp.mpf('0.7501')]
for i in range(0,100):
    x.append(4*x[i]*(1-x[i]))

mpmath.mp.dps=500
xl=[mpmath.mp.mpf('0.7501')]
for i in range(0,100):
    xl.append(4*xl[i]*(1-xl[i]))

reldiff_x=[mpmath.fabs((xl[i]-x[i])/xl[i])for i in range(len(x))]
print('    i,                   x[i]                    ,reldiff_x[i]')
for i in range(0,101):
    if i%10==0:
        print(f'{i:5d}',mpmath.nstr(x[i],40),',',mpmath.nstr(reldiff_x[i],2))

mpmath.libmp.BACKEND =  python
    i,                   x[i]                    ,reldiff_x[i]
    0 0.7501 , 6.2e-353
   10 0.844495953602217447537148702561541372674 , 1.7e-349
   20 0.1429397245123076552842817572313062830738 , 1.0e-345
   30 0.854296003704421891666131184258887443748 , 1.8e-343
   40 0.7749757531182012412802234612642116848151 , 2.4e-340
   50 0.09337533219770302905518966801516823476282 , 1.4e-336
   60 0.4082201682908781318710276618195268673036 , 5.5e-334
   70 0.07151199970505857489709934641759321833017 , 1.7e-330
   80 0.4632533029007757105599346745832074306278 , 5.2e-328
   90 0.001334405012086884046469201214999119068328 , 1.3e-323
  100 0.07881798937150990680670477046269926472401 , 1.7e-321


In [5]:
#課題6
import gmpy2

gmpy2.get_context().precision=128
x=[gmpy2.mpfr('0.7501')]
for i in range(0,100):
    x.append(4*x[i]*(1-x[i]))

gmpy2.get_context().precision=128
xl=[gmpy2.mpfr('0.7501')]
for i in range(0,100):
    xl.append(4*xl[i]*(1-xl[i]))

reldiff_x=[gmpy2.reldiff(xl[i],x[i])for i in range(len(x))]
print('    i,            x[i]')
for i in range(0,101):
    if i%10==0:
        print(f'{i:5d},{x[i]:50.40e},{reldiff_x[i]:5.1e}')

    i,            x[i]
    0,    7.5009999999999999999999999999999999999957e-01,0.0e+00
   10,    8.4449595360221744753714870256154137269870e-01,0.0e+00
   20,    1.4293972451230765528428175723130626119287e-01,0.0e+00
   30,    8.5429600370442189166613118425886484760369e-01,0.0e+00
   40,    7.7497575311820124128022346123682417226484e-01,0.0e+00
   50,    9.3375332197703029055189687555128488838985e-02,0.0e+00
   60,    4.0822016829087813187106146216161441532349e-01,0.0e+00
   70,    7.1511999705058574878953745604654169664485e-02,0.0e+00
   80,    4.6325330290077567460258670989545944685968e-01,0.0e+00
   90,    1.3344050120841884955920667169676700883139e-03,0.0e+00
  100,    7.8817989391884034027630801238868292349517e-02,0.0e+00


In [1]:
#課題4
import mpmath
print('mpmath.libmp.BACKEND = ',mpmath.libmp.BACKEND)

mpmath.libmp.BACKEND =  gmpy


In [8]:
#課題7
# rmode.py: 丸めモード変更
# [Windows&Linux]
#  https://rafaelbarreto.wordpress.com/2009/03/30/controlling-fpu-rounding-modes-with-python/
# [macOS]
#  https://stackoverflow.com/questions/16000574/tie-breaking-of-round-with-numpy

# OS種別
# https://stackoverflow.com/questions/1854/python-what-os-am-i-running-on
import platform

# DLL呼び出し
# from ctypes import cdll
from ctypes import *

# 丸めモード定義
global FE_TOZERO, FE_DOWNWARD, FE_UPWARD, FE_TONEAREST
# set_rmode(丸めモード), 丸めモード = get_rmode()
global set_rmode, get_rmode

# OS名
os_name = platform.system()

# Windows
if os_name == 'Windows':
    msvcrt = cdll.msvcrt
    MW_RC = 0x00000300

    def set_rmode(mode):
        msvcrt._controlfp_s(0, mode, MW_RC)

    FE_TOZERO = 0x00000300
    FE_DOWNWARD = 0x00000100
    FE_UPWARD = 0x00000200
    FE_TONEAREST = 0x00000000

    def get_rmode():
        current_rmode = c_uint()
        msvcrt._controlfp_s(byref(current_rmode), 0, 0)

        return current_rmode.value & MW_RC

# Linux
elif os_name == 'Linux':
    from ctypes.util import find_library
    libm = cdll.LoadLibrary(find_library('m'))
    set_rmode, get_rmode = libm.fesetround, libm.fegetround
    # x86
    FE_TOZERO = 0xc00
    FE_DOWNWARD = 0x400
    FE_UPWARD = 0x800
    FE_TONEAREST = 0
# macOS
elif os_name == "Darwin":
    from ctypes.util import find_library
    libc = cdll.LoadLibrary(find_library('c'))
    set_rmode, get_rmode = libc.fesetround, libc.fegetround
    # x86
    FE_TOZERO = 0xc00
    FE_DOWNWARD = 0x400
    FE_UPWARD = 0x800
    FE_TONEAREST = 0


# 丸めモード確認
def print_rmode():
    rmode = get_rmode()

    if rmode == FE_TOZERO:
        print('切り捨て')
    elif rmode == FE_DOWNWARD:
        print('-Infへの丸め')
    elif rmode == FE_UPWARD:
        print('+Infへの丸め')
    elif rmode == FE_TONEAREST:
        print('最近偶数値丸め')
    else:
        print('不明 ', rmode)

    return rmode


# -------------------------------------
# Copyright (c) 2021 Tomonori Kouya
# All rights reserved.
# -------------------------------------

import numpy as np

a=0.7501
b=22.0
c=3.0
print('rmode')
d=b**2-4*a*c
print('ax^2+bx+c=0の解は')
print('x=',(-1*b+np.sqrt(d))/(2*a))
print('x=',(-1*b-np.sqrt(d))/(2*a))

rmode
ax^2+bx+c=0の解は
x= -0.13700360765370165
x= -29.192419135980483


In [9]:
# interval.py: 区間演算クラス
# R.T.Kneusel, "Numbers and Computers", Springer, 2015.
# Interval -> [left, right]
# rmode.py: 丸めモード変更
# [Windows&Linux]
#  https://rafaelbarreto.wordpress.com/2009/03/30/controlling-fpu-rounding-modes-with-python/
# [macOS]
#  https://stackoverflow.com/questions/16000574/tie-breaking-of-round-with-numpy

# OS種別
# https://stackoverflow.com/questions/1854/python-what-os-am-i-running-on
import platform

# DLL呼び出し
# from ctypes import cdll
from ctypes import *

# 丸めモード定義
global FE_TOZERO, FE_DOWNWARD, FE_UPWARD, FE_TONEAREST
# set_rmode(丸めモード), 丸めモード = get_rmode()
global set_rmode, get_rmode

# OS名
os_name = platform.system()

# Windows
if os_name == 'Windows':
    msvcrt = cdll.msvcrt
    MW_RC = 0x00000300

    def set_rmode(mode):
        msvcrt._controlfp_s(0, mode, MW_RC)

    FE_TOZERO = 0x00000300
    FE_DOWNWARD = 0x00000100
    FE_UPWARD = 0x00000200
    FE_TONEAREST = 0x00000000

    def get_rmode():
        current_rmode = c_uint()
        msvcrt._controlfp_s(byref(current_rmode), 0, 0)

        return current_rmode.value & MW_RC

# Linux
elif os_name == 'Linux':
    from ctypes.util import find_library
    libm = cdll.LoadLibrary(find_library('m'))
    set_rmode, get_rmode = libm.fesetround, libm.fegetround
    # x86
    FE_TOZERO = 0xc00
    FE_DOWNWARD = 0x400
    FE_UPWARD = 0x800
    FE_TONEAREST = 0
# macOS
elif os_name == "Darwin":
    from ctypes.util import find_library
    libc = cdll.LoadLibrary(find_library('c'))
    set_rmode, get_rmode = libc.fesetround, libc.fegetround
    # x86
    FE_TOZERO = 0xc00
    FE_DOWNWARD = 0x400
    FE_UPWARD = 0x800
    FE_TONEAREST = 0


# 丸めモード確認
def print_rmode():
    rmode = get_rmode()

    if rmode == FE_TOZERO:
        print('切り捨て')
    elif rmode == FE_DOWNWARD:
        print('-Infへの丸め')
    elif rmode == FE_UPWARD:
        print('+Infへの丸め')
    elif rmode == FE_TONEAREST:
        print('最近偶数値丸め')
    else:
        print('不明 ', rmode)

    return rmode


# -------------------------------------
# Copyright (c) 2021 Tomonori Kouya
# All rights reserved.
# -------------------------------------

import math


class Interval:

    # 開始前のデフォルト丸めモード
    default_rmode = get_rmode()

    # コンストラクタ
    def __init__(self, left, right=None):
        if right is None:
            self.left = left
            self.right = left
        else:
            self.left = left
            self.right = right

    # + : 加算
    def __add__(self, y):
        set_rmode(FE_DOWNWARD)
        left = self.left + y.left
        set_rmode(FE_UPWARD)
        right = self.right + y.right
        set_rmode(Interval.default_rmode)

        return Interval(left, right)

    # - : 減算
    def __sub__(self, y):
        set_rmode(FE_DOWNWARD)
        left = self.left - y.right
        set_rmode(FE_UPWARD)
        right = self.right - y.left
        set_rmode(Interval.default_rmode)

        return Interval(left, right)

    # - :マイナス
    def __neg__(self):
        left = -self.right
        right = -self.left

        return Interval(left, right)

    # * : 乗算
    def __mul__(self, y):
        if not isinstance(y, Interval):
            y = Interval(float(y))
        set_rmode(FE_DOWNWARD)
        left = min([
            self.left * y.left,
            self.right * y.left,
            self.left * y.right,
            self.right * y.right
        ])
        set_rmode(FE_UPWARD)
        right = max([
            self.left * y.left,
            self.right * y.left,
            self.left * y.right,
            self.right * y.right
        ])
        set_rmode(Interval.default_rmode)

        return Interval(left, right)

    __rmul__ = __mul__

    # recip(x) = 1/x: 逆数
    def recip(self):
        if (self.left < 0.0) and (self.right > 0.0):
            return Interval(-math.inf, math.inf)

        set_rmode(FE_DOWNWARD)
        left = 1.0 / self.right
        set_rmode(FE_UPWARD)
        right = 1.0 / self.left
        set_rmode(Interval.default_rmode)

        return Interval(left, right)

    # / : 除算
    def __truediv__(self, y):
        return self.__mul__(y.recip())

    # 絶対値
    def abs(self):
        return max([math.abs(self.left), math.abs(self.right)])

    # 平方根
    def sqrt(self):
        if (self.left < 0) or (self.right < 0):
            return Interval(-math.nan, -math.nan)

        return Interval(math.sqrt(self.left), math.sqrt(self.right))

    # ** : べき乗
    def __pow__(self, n):
        if (n % 2) == 1:
            set_rmode(FE_DOWNWARD)
            left = self.left ** n
            set_rmode(FE_UPWARD)
            right = self.right ** n
        elif self.left >= 0:
            set_rmode(FE_DOWNWARD)
            left = self.left ** n
            set_rmode(FE_UPWARD)
            right = self.right ** n
        elif self.right < 0:
            set_rmode(FE_DOWNWARD)
            left = self.right ** n
            set_rmode(FE_UPWARD)
            right = self.left ** n
        else:
            left = 0.0
            set_rmode(FE_UPWARD)
            right = self.left ** n
            temp = self.right ** n
            if temp > right:
                right = temp

        set_rmode(Interval.default_rmode)
        return Interval(left, right)

    # 出力用
    def __str__(self):
        return "[ %g, %g ]" % (self.left, self.right)

    __repr__ = __str__


# -------------------------------------
# Copyright (c) 2021 Tomonori Kouya
# All rights reserved.
# -------------------------------------
import numpy as np

a=0.7501
b=22.0
c=3.0
print('rmode')
d=b**2-4*a*c
print('ax^2+bx+c=0の解は')
print('x=',(-1*b+np.sqrt(d))/(2*a))
print('x=',(-1*b-np.sqrt(d))/(2*a))

rmode
ax^2+bx+c=0の解は
x= -0.13700360765370165
x= -29.192419135980483


In [10]:
import mpmath

print('mpmath.libmp.BACKEND = ',mpmath.libmp.BACKEND)

mpmath.mp.dps=350
a=0.7501
b=22.0
c=3.0
print('rmode')
d=b**2-4*a*c
print('ax^2+bx+c=0の解は')
print('x=',(-1*b+np.sqrt(d))/(2*a))
print('x=',(-1*b-np.sqrt(d))/(2*a))

mpmath.libmp.BACKEND =  gmpy
rmode
ax^2+bx+c=0の解は
x= -0.13700360765370165
x= -29.192419135980483


In [11]:
import gmpy2

gmpy2.get_context().precision=128
a=0.7501
b=22.0
c=3.0
print('rmode')
d=b**2-4*a*c
print('ax^2+bx+c=0の解は')
print('x=',(-1*b+np.sqrt(d))/(2*a))
print('x=',(-1*b-np.sqrt(d))/(2*a))

rmode
ax^2+bx+c=0の解は
x= -0.13700360765370165
x= -29.192419135980483


In [12]:
#課題8
import mpmath

print('mpmath')

mpmath.mp.dps=350
x=[mpmath.mp.mpf('0.7501')]
for i in range(0,1000):
    x.append(4*x[i]*(1-x[i]))

mpmath.mp.dps=500
xl=[mpmath.mp.mpf('0.7501')]
for i in range(0,1000):
    xl.append(4*xl[i]*(1-xl[i]))

reldiff_x=[mpmath.fabs((xl[i]-x[i])/xl[i])for i in range(len(x))]
print('    i,                   x[i]                    ,reldiff_x[i]')
for i in range(0,1001):
    if i%100==0:
        print(f'{i:5d}',mpmath.nstr(x[i],40),',',mpmath.nstr(reldiff_x[i],2))
        
import gmpy2

print('gmpy2')

gmpy2.get_context().precision=128
x=[gmpy2.mpfr('0.7501')]
for i in range(0,1000):
    x.append(4*x[i]*(1-x[i]))

gmpy2.get_context().precision=128
xl=[gmpy2.mpfr('0.7501')]
for i in range(0,1000):
    xl.append(4*xl[i]*(1-xl[i]))

reldiff_x=[gmpy2.reldiff(xl[i],x[i])for i in range(len(x))]
print('    i,            x[i]')
for i in range(0,1001):
    if i%100==0:
        print(f'{i:5d},{x[i]:50.40e},{reldiff_x[i]:5.1e}')

mpmath
    i,                   x[i]                    ,reldiff_x[i]
    0 0.7501 , 6.2e-353
  100 0.07881798937150990680670477046269926472401 , 1.7e-321
  200 0.1061630592714161938954317714188532071936 , 1.9e-291
  300 0.1925556772153730403309029871322957065354 , 1.7e-261
  400 0.737610785581809526191915650677174162771 , 6.1e-232
  500 0.2510871563169473294794265669495846136844 , 2.2e-201
  600 0.5575707689493004231099009436351827165796 , 1.5e-171
  700 0.8243380130413461181716339173423200307025 , 9.6e-142
  800 0.5552764070958395874216157638791411213167 , 2.4e-111
  900 0.8480060542220386188777169502268659662631 , 1.4e-81
 1000 0.3537546114527867747262863983124888337521 , 5.8e-51
gmpy2
    i,            x[i]
    0,    7.5009999999999999999999999999999999999957e-01,0.0e+00
  100,    7.8817989391884034027630801238868292349517e-02,0.0e+00
  200,    3.5486002071777466298019378673684047503469e-01,0.0e+00
  300,    5.2968070776341605605703654576258405398797e-01,0.0e+00
  400,    9.9924205